In [12]:
import langchain 

In [13]:
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="RAG-project"


In [14]:
from langsmith import Client
client=Client()

In [15]:
#CALL LLM
from langchain_ollama import OllamaLLM
llm=OllamaLLM(model="llama3")
#llm_response=llm.invoke("Tell me a joke")
#llm_response


In [16]:
#parsing output 

In [17]:
#simple chain

In [18]:
#structured output

In [19]:
#prompt template

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 

#Define the prompt 
prompt=ChatPromptTemplate.from_template("Tell me a joke about {topic}") # string with placeholder
#Initialize the LLM
llm=OllamaLLM(model="llama3")
#Define the output parser
output_parser=StrOutputParser()
#compose the chain
chain=prompt | llm | output_parser
#Use the chain
#result=chain.invoke({"topic":"programming"})
#print(result)

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage,SystemMessage

system_message=SystemMessage(content="You are a helpful assistant that tells jokes.")
human_message=HumanMessage(content="Tell me about programming")
llm.invoke([system_message,human_message])

"Programming! It's like baking a cake, but instead of flour and sugar, you use code and caffeine!\n\nBut seriously, programming is the art of creating instructions that computers can understand. Think of it like writing a recipe for your computer to follow. And just like how you need to have the right ingredients in the right order to make a delicious cake, you need to have the right code in the right order to create a program!\n\nSpeaking of recipes, here's one: Why did the programmer quit his job? Because he didn't get arrays! (get it?)\n\nSo, if you're interested in learning more about programming, I'd be happy to help you start coding your own recipe book... er, programs!"

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
template= ChatPromptTemplate([
    ("system","you are a helpful assistant that tells jokes."),
    ("human","tell me about{topic}")
])
prompt_value=template.invoke(
    {
        "topic":"programming"
    }
)
prompt_value


ChatPromptValue(messages=[SystemMessage(content='you are a helpful assistant that tells jokes.', additional_kwargs={}, response_metadata={}), HumanMessage(content='tell me aboutprogramming', additional_kwargs={}, response_metadata={})])

In [22]:
#llm.invoke(prompt_value)

In [23]:
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from typing import List
from langchain_core.documents import Document

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=400,
    length_function=len
)
pdf_loader=PyPDFLoader("./documents/Ayurvedic Dermatology Therapeutic Strategies.pdf")
documents=pdf_loader.load()
print(len(documents))
splits=text_splitter.split_documents(documents)
print(f"Split the document into {len(splits) } chunks")


4
Split the document into 17 chunks


In [24]:
documents[0]

Document(metadata={'source': './documents/Ayurvedic Dermatology Therapeutic Strategies.pdf', 'page': 0, 'page_label': '1'}, page_content=' \nInternational Journal for Multidisciplinary Research (IJFMR) \n \nE-ISSN: 2582-2160   ●   Website: www.ijfmr.com       ●   Email: editor@ijfmr.com \n \nIJFMR23057259 Volume 5, Issue 5, September-October 2023 1 \n \nAyurvedic Dermatology: Therapeutic Strategies \nfor Skin Disease Management \n \nDr.Bhupendra Parihar1, Dr.Pravanjan Acharya2, Dr.Vivek Shrivastava3 \n \n1M.D. Scholar, Department of Samhita Siddhant \n2M.D, Ph.D, H.O.D and Professor, Department of Samhita Siddhant , Govt. Ayurveda College And \nHospital , Rewa  (M.P) \n3M.D, Assistant Professor, Department of Samhita Siddhant, Govt. Ayurveda College and Hospital \nRewa (M.P) \n \nABSTRACT  \nHuman skin, the outer layer of the body, is the largest organ in the body. It is also the first line of defense. \nAccording to Ayurveda, the skin is one of the most important sensory organs. Ayurv

In [25]:
splits[0]

Document(metadata={'source': './documents/Ayurvedic Dermatology Therapeutic Strategies.pdf', 'page': 0, 'page_label': '1'}, page_content='International Journal for Multidisciplinary Research (IJFMR) \n \nE-ISSN: 2582-2160   ●   Website: www.ijfmr.com       ●   Email: editor@ijfmr.com \n \nIJFMR23057259 Volume 5, Issue 5, September-October 2023 1 \n \nAyurvedic Dermatology: Therapeutic Strategies \nfor Skin Disease Management \n \nDr.Bhupendra Parihar1, Dr.Pravanjan Acharya2, Dr.Vivek Shrivastava3 \n \n1M.D. Scholar, Department of Samhita Siddhant \n2M.D, Ph.D, H.O.D and Professor, Department of Samhita Siddhant , Govt. Ayurveda College And \nHospital , Rewa  (M.P) \n3M.D, Assistant Professor, Department of Samhita Siddhant, Govt. Ayurveda College and Hospital \nRewa (M.P) \n \nABSTRACT  \nHuman skin, the outer layer of the body, is the largest organ in the body. It is also the first line of defense. \nAccording to Ayurveda, the skin is one of the most important sensory organs. Ayurveda

In [26]:
#Function to load documents from a folder
def load_documents(folder_path:str)->List[Document]:
    documents=[]
    for filename in os.listdir(folder_path):
        file_path=os.path.join(folder_path,filename)
        if filename.endswith('.pdf'):
            loader=PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader=Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

#Load documents from a folder
folder_path="./documents"
documents=load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder")
splits=text_splitter.split_documents(documents)
print(f"splits documents into {len(splits)} chunks")

Loaded 20 documents from the folder
splits documents into 138 chunks


In [27]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
documents_embeddings=embedding_function.embed_documents([split.page_content for split in splits])
documents_embeddings[0]



C:\Users\Vidhi\AppData\Local\Temp\ipykernel_17072\154255307.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\Vidhi\Desktop\RAG test\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[-0.010292882099747658,
 0.009824150241911411,
 -0.01196305826306343,
 0.045850496739149094,
 0.034808434545993805,
 0.03787285089492798,
 0.0007567503489553928,
 0.07701867818832397,
 0.005133001133799553,
 0.04649583250284195,
 0.07154428958892822,
 -0.02949429675936699,
 -0.006712678819894791,
 0.07181013375520706,
 0.03831636905670166,
 -0.016130225732922554,
 0.017216026782989502,
 -0.030118396505713463,
 0.0025587868876755238,
 -0.053622130304574966,
 -0.04071718454360962,
 0.04429784044623375,
 0.002426828257739544,
 -0.08726373314857483,
 -0.04836170747876167,
 -0.08122863620519638,
 -0.005389536265283823,
 -0.005218025762587786,
 -0.03237287700176239,
 -0.08951034396886826,
 0.009979157708585262,
 -0.028474414721131325,
 0.012854800559580326,
 -0.01571258157491684,
 -0.14818677306175232,
 0.029052039608359337,
 -0.07276295125484467,
 -0.009471915662288666,
 0.030162619426846504,
 0.029259417206048965,
 -0.0694456398487091,
 0.025118818506598473,
 -0.05138091370463371,
 -0.0427

In [28]:
from langchain.vectorstores import Chroma
embedding_function=HuggingFaceEmbeddings()
collection_name="my_collection"
vectorstore=Chroma.from_documents(collection_name=collection_name,documents=splits,embedding=embedding_function,persist_directory="./chroma_db")
print("Vector store created and persisted to './chroma_db'")


C:\Users\Vidhi\AppData\Local\Temp\ipykernel_17072\43092270.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_function=HuggingFaceEmbeddings()


Vector store created and persisted to './chroma_db'


In [29]:
#perform similarity search 
query="Name the pattern of skin diseases"
search_results=vectorstore.similarity_search(query,k=2)
print(f"\n Top 2 most relevent chunks for the query: '{query} \n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


 Top 2 most relevent chunks for the query: 'Name the pattern of skin diseases 

Result 1:
Source: ./documents\Ayurvedic Dermatology Therapeutic Strategies.pdf
Content: manifestation and/or aggravation of many dermatological diseases), etc. 
The incidence of skin diseases is increasing due to the following factors: 
⮚  Unappropriate daily diet and lifestyle; Eating intolerable and unhealthy foods; Eating before the 
previous meal has been digested; take milk with meat, fish (aquatic animals); bathe immediately after 
sunbathing or after physical activity), etc.

Result 2:
Source: ./documents\Ayurvedic Dermatology Therapeutic Strategies.pdf
Content: a wide spectrum of etiological factors of dermatological diseases.  Etiological factors include physical, 
physiological, hereditary, psychological, psychosocial and papkarma factors (sinful 
activities/psychosocial stress and related psychological factors are mainly responsible for the 
manifestation and/or aggravation of many dermatologica

In [30]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke("Name the pattern of skin diseases")
print(retriever_results)

[Document(metadata={'page': 0, 'page_label': '1', 'source': './documents\\Ayurvedic Dermatology Therapeutic Strategies.pdf'}, page_content='manifestation and/or aggravation of many dermatological diseases), etc. \nThe incidence of skin diseases is increasing due to the following factors: \n⮚  Unappropriate daily diet and lifestyle; Eating intolerable and unhealthy foods; Eating before the \nprevious meal has been digested; take milk with meat, fish (aquatic animals); bathe immediately after \nsunbathing or after physical activity), etc.'), Document(metadata={'page': 0, 'page_label': '1', 'source': './documents\\Ayurvedic Dermatology Therapeutic Strategies.pdf'}, page_content='a wide spectrum of etiological factors of dermatological diseases.  Etiological factors include physical, \nphysiological, hereditary, psychological, psychosocial and papkarma factors (sinful \nactivities/psychosocial stress and related psychological factors are mainly responsible for the \nmanifestation and/or ag

In [31]:
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)


In [32]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain=(
    {"context":retriever, "question":RunnablePassthrough()} | prompt
)
#rag_chain.invoke("Name the pattern of skin diseases")

In [33]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain=(
    {"context":retriever | docs2str, "question":RunnablePassthrough()} | prompt
)
#rag_chain.invoke("Name the pattern of skin diseases")



In [36]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question="Name the pattern of skin diseases"
response=rag_chain.invoke(question)
print(response)

According to the context, the pattern of skin diseases mentioned is:

"many dermatological diseases"

This does not specify a particular type or group of skin diseases, but rather implies that there are various types of skin diseases that can be caused by the listed factors.


In [37]:
#chat history
from langchain_core.messages import  HumanMessage,AIMessage
chat_history=[]
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

In [38]:
chat_history


[HumanMessage(content='Name the pattern of skin diseases', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the context, the pattern of skin diseases mentioned is:\n\n"many dermatological diseases"\n\nThis does not specify a particular type or group of skin diseases, but rather implies that there are various types of skin diseases that can be caused by the listed factors.', additional_kwargs={}, response_metadata={})]

In [39]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
print(contextualize_chain.invoke({"input": "Explain one of them", "chat_history": chat_history}))

AI Reformulation:
Explain one of the many dermatological diseases.

(Note: The original question already implies that there is a list of skin diseases, so no additional context is needed to understand this standalone question.)


In [40]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
history_aware_retriever.invoke({"input": "Explain one of them", "chat_history": chat_history})

[Document(metadata={'page': 1, 'page_label': '708', 'source': './documents\\skin diseases.pdf'}, page_content='the members of two rural communities claimed to have a skin\ndisease (Figueroa and others 1998), but when they were exam-\nined, 67 percent of those who denied having skin problems\nwere found to have treatable skin conditions, most of which\nwere infections. However, prevalence alone does not equate\nwith disease burden. For instance, most communities recognize\nscabies as a problem because of its intractable itching and sec-\nondary infection, whereas they may ignore tinea capitis, which'),
 Document(metadata={'page': 0, 'page_label': '1', 'source': './documents\\Ayurvedic Dermatology Therapeutic Strategies.pdf'}, page_content='a wide spectrum of etiological factors of dermatological diseases.  Etiological factors include physical, \nphysiological, hereditary, psychological, psychosocial and papkarma factors (sinful \nactivities/psychosocial stress and related psychological 

In [41]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
rag_chain.invoke({"input": "Explain one of them", "chat_history": chat_history})

{'input': 'Explain one of them',
 'chat_history': [HumanMessage(content='Name the pattern of skin diseases', additional_kwargs={}, response_metadata={}),
  AIMessage(content='According to the context, the pattern of skin diseases mentioned is:\n\n"many dermatological diseases"\n\nThis does not specify a particular type or group of skin diseases, but rather implies that there are various types of skin diseases that can be caused by the listed factors.', additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'page': 0, 'page_label': '707', 'source': './documents\\skin diseases.pdf'}, page_content='and war-related injuries. As noted later, those figures require\nconfirmation by more detailed studies, and their practical\napplication to health interventions needs to be tested.\nAssessing the impact of skin disease on the quality of life in\ncomparison with that of chronic nondermatological diseases is\ndifficult; however, the study by Mallon and others (1999),\nwhich

In [42]:
from pymongo import MongoClient
DB_URI = "mongodb://localhost:27017/"  
DB_NAME = "rag_app"
def get_db_connection():
    client = MongoClient(DB_URI)
    db = client[DB_NAME]
    return db


def create_application_logs():
    db = get_db_connection()
    # MongoDB creates collections automatically when inserting the first document.
    # Here we explicitly create an index for better query performance.
    db.application_logs.create_index("session_id")


def insert_application_logs(session_id, user_query, gpt_response, model):
    db = get_db_connection()
    log_entry = {
        "session_id": session_id,
        "user_query": user_query,
        "gpt_response": gpt_response,
        "model": model,
        "created_at": datetime.utcnow()
    }
    db.application_logs.insert_one(log_entry)


def get_chat_history(session_id):
    db = get_db_connection()
    logs = db.application_logs.find({"session_id": session_id}).sort("created_at", 1)
    messages = []
    for log in logs:
        messages.extend([
            {"role": "human", "content": log['user_query']},
            {"role": "ai", "content": log['gpt_response']}
        ])
    return messages




In [43]:
import uuid
from datetime import datetime
# Example usage for a new user
session_id = str(uuid.uuid4())
question = "Name the skin diseases"
chat_history = get_chat_history(session_id)  # Fetch chat history from MongoDB
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']

# Insert the new interaction into MongoDB
insert_application_logs(session_id, question, answer, "llama3")

# Print the interaction
print(f"Human: {question}")
print(f"AI: {answer}\n")

# Example of a follow-up question
question2 = "Describe one in detail"
chat_history = get_chat_history(session_id)  # Fetch updated chat history
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']

# Insert the follow-up interaction into MongoDB
insert_application_logs(session_id, question2, answer2, "llama3")

# Print the interaction
print(f"Human: {question2}")
print(f"AI: {answer2}")


C:\Users\Vidhi\AppData\Local\Temp\ipykernel_17072\88342757.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow()


Human: Name the skin diseases
AI: Based on the provided context, the following skin diseases were mentioned:

1. Pyoderma
2. Pediculosis
3. Eczema or dermatitis
4. HIV-related skin disease
5. Pigmentary anomalies (including hypopigmentation, melasma, and dermatitis caused by cosmetic bleaching agents)
6. Acne



C:\Users\Vidhi\AppData\Local\Temp\ipykernel_17072\88342757.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow()


Human: Describe one in detail
AI: Based on the provided context, I'll describe Eczema or Dermatitis in detail:

Eczema or Dermatitis is a skin disease that can significantly impact an individual's quality of life. It is characterized by dry, itchy, and inflamed skin, which can lead to disfigurement, disability, and reduction in overall well-being.

The symptoms of eczema or dermatitis include:

* Dry, scaly, or flaky skin
* Redness and inflammation
* Intense itching, which can be severe enough to disrupt daily activities
* Crusting and oozing, especially if the condition is left untreated

Eczema or dermatitis can occur anywhere on the body, but it commonly affects areas such as the face, arms, legs, and torso. The disease can be caused by a combination of genetic, environmental, and lifestyle factors.

In this context, eczema or dermatitis is mentioned alongside other skin diseases that pose significant morbidity and impact quality of life. It highlights the importance of screening fo